In [2]:
from pyspark.sql import SparkSession


# Initialize PySpark with MongoDB and Elastic support
spark = (SparkSession.builder.appName("Collecting and Displaying Records")
    # Load support for MongoDB and Elasticsearch
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,org.elasticsearch:elasticsearch-spark-30_2.12:7.14.2")
    # Add Configuration for MongopDB
    .config("spark.mongodb.input.uri", "mongodb://mongo:27017/test.coll")
    .config("spark.mongodb.output.uri", "mongodb://mongo:27017/test.coll")
    .getOrCreate())
sc = spark.sparkContext
sc.setLogLevel("ERROR")

print("\nPySpark initialized...")

22/02/14 23:47:18 WARN Utils: Your hostname, kellyui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
22/02/14 23:47:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/kelly/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/kelly/.ivy2/cache
The jars for the packages stored in: /Users/kelly/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0164edb2-cb76-4c88-8642-154a5b218fca;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
	found org.elasticsearch#elasticsearch-spark-30_2.12;7.14.2 in central
	found org.scala-lang#scala-reflect;2.12.8 in central
	found org.slf4j#slf4j-api;1.7.6 in central
	found commons-logging#commons-logging;1.1.1 in central
	found javax.xml.bind#jaxb-api;2.3.1 in central
	found com.google.protobuf#protobuf-java;2.5.0 in central
	found org.apache.spark#spark-yarn_2.12;3.0.1 in central
:: resolution repo


PySpark initialized...


In [ ]:
on_time_df = spark.read.csv('data/On_Time_On_Time_Performance_2015.csv.bz2', header=True)
on_time_df.limit(20).toPandas()

In [ ]:
on_time_df.registerTempTable("raw_on_time_performance")

trimmed_cast_performance = spark.sql("""
SELECT
  Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate,
  Carrier, TailNum, FlightNum,
  Origin, OriginCityName, OriginState,
  Dest, DestCityName, DestState,
  DepTime, cast(DepDelay as float), cast(DepDelayMinutes as int),
  cast(TaxiOut as float), cast(TaxiIn as float),
  WheelsOff, WheelsOn,
  ArrTime, cast(ArrDelay as float), cast(ArrDelayMinutes as float),
  cast(Cancelled as int), cast(Diverted as int),
  cast(ActualElapsedTime as float), cast(AirTime as float),
  cast(Flights as int), cast(Distance as float),
  cast(CarrierDelay as float), cast(WeatherDelay as float), 
  cast(NASDelay as float),
  cast(SecurityDelay as float), 
  cast(LateAircraftDelay as float),
  CRSDepTime, CRSArrTime
FROM
  raw_on_time_performance
""")

trimmed_cast_performance.registerTempTable("on_time_performance")
trimmed_cast_performance.limit(10).toPandas().head()